# 변경점

- data augmentation강화 flip류를 제외하고, 블러등 인공적인 노이즈말고, 이미지의 색영역을 변경하는 augmentation을 Oneof method로 2가지 구현
- 이미지의 feature extracting을 더 강화하기 위해 해상도를 높여서 신경망에 투입(224->400)

## 1. Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

## 2. Device

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## 3. Hyperparameter

In [3]:
CFG = {
    'IMG_SIZE':400,
    'EPOCHS':70,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

## 4. Fixed Randomseed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
all_img_list = glob.glob('./data/open/train/*/*')

In [6]:
print(all_img_list[2])

./data/open/train/터짐/72.png


In [7]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [8]:
df

,img_path,label
0,./data/open/train/터짐/27.png,터짐
1,./data/open/train/터짐/114.png,터짐
2,./data/open/train/터짐/72.png,터짐
3,./data/open/train/터짐/160.png,터짐
4,./data/open/train/터짐/13.png,터짐
...,...,...
3452,./data/open/train/녹오염/0.png,녹오염
3453,./data/open/train/녹오염/8.png,녹오염
3454,./data/open/train/녹오염/11.png,녹오염
3455,./data/open/train/녹오염/10.png,녹오염


In [9]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [10]:
train

,img_path,label
1562,./data/open/train/훼손/689.png,훼손
329,./data/open/train/훼손/112.png,훼손
561,./data/open/train/훼손/943.png,훼손
839,./data/open/train/훼손/1214.png,훼손
545,./data/open/train/훼손/981.png,훼손
...,...,...
1185,./data/open/train/훼손/106.png,훼손
2243,./data/open/train/오염/533.png,오염
3341,./data/open/train/면불량/86.png,면불량
1007,./data/open/train/훼손/460.png,훼손


## 5. Label-Encoding

In [11]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [12]:
train

,img_path,label
1562,./data/open/train/훼손/689.png,18
329,./data/open/train/훼손/112.png,18
561,./data/open/train/훼손/943.png,18
839,./data/open/train/훼손/1214.png,18
545,./data/open/train/훼손/981.png,18
...,...,...
1185,./data/open/train/훼손/106.png,18
2243,./data/open/train/오염/533.png,10
3341,./data/open/train/면불량/86.png,6
1007,./data/open/train/훼손/460.png,18


In [13]:
train['label'].value_counts()

label
18    983
10    416
1     215
3     147
15    113
2     102
11     99
7      91
6      69
9      40
5      38
17     36
14     19
12     15
13     12
4      10
0       8
16      4
8       2
Name: count, dtype: int64

## 6. CustomDataset Define

In [14]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

## 7. Data Augmetation

In [15]:
# train_transform = A.Compose([
#                             A.Resize(240,240),
#                             A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.OneOf([
#                               A.HorizontalFlip(p=1),
#                               A.RandomRotate90(p=1),
#                               A.VerticalFlip(p=1),
#                               A.Affine(rotate=(-45, 45),scale=(0.7,0.9),p=1),
#                             ], p=1),
#                             A.OneOf([
#                               A.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), p=1),
#                               A.RandomBrightnessContrast (brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
#                               A.ChannelShuffle(p=1),
#                               A.RGBShift (r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),
#                             ], p=0.5),
#                             ToTensorV2()
#                             ])

In [16]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE']+10,CFG['IMG_SIZE']+10),
                            A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.4),
                            A.OneOf([
                              A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1),
                              A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
                              A.ChannelShuffle(p=1),
                              A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),
                            ], p=0.5),
                            A.OneOf([
                              A.ElasticTransform(p=1),
                              A.Emboss(p=1),
                              A.HueSaturationValue(p=1),
                            ], p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [17]:
# train_transform = A.Compose([
#                             A.Resize(240,240),
#                             A.RandomCrop(224, 224),
#                             A.OneOf([
#                               A.HorizontalFlip(p=1),
#                               A.RandomRotate90(p=1),
#                               A.VerticalFlip(p=1)            
#                             ], p=1),
#                             A.OneOf([
#                               A.MotionBlur(p=1),
#                               A.OpticalDistortion(p=1),
#                               A.GaussNoise(p=1)                 
#                             ], p=1),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])

In [18]:
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

## 8. Data Loader

In [19]:
# Data label imbalance할때 dataloader에서 weightrandomsampler를 통해 균형있게 데이터를 loade하는 방법
# https://rueki.tistory.com/244 참고

# label_counts = [a_list[i] for i in range(len(class_counts))]
a_list = train['label'].value_counts()
class_counts = train['label'].value_counts().to_list()
class_weights = [1 / a_list[i] for i in range(len(class_counts))]

num_samples = sum(class_counts)
labels_lists = train['label'].to_list()
weights = [class_weights[labels_lists[i]] for i in range(int(num_samples))]

sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples), replacement=True)

In [20]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], sampler = sampler, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## 9. Model Define

In [21]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.densenet161(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [22]:
# class BaseModel(nn.Module):
#     def __init__(self, num_classes=len(le.classes_)):
#         super(BaseModel, self).__init__()
#         self.backbone = models.vit_l_32(pretrained=True)
#         self.classifier = nn.Linear(1000, num_classes)
        
#     def forward(self, x):
#         x = self.backbone(x)
#         x = self.classifier(x)
#         return x

### 1) train

In [23]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

### 2) validation

In [24]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## 10. Model train

In [25]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-9, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.61270] Val Loss : [1.66389] Val Weighted F1 Score : [0.67397]


  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

...

  0%|          | 0/65 [00:00<?, ?it/s]

Epoch [69], Train Loss : [0.60466] Val Loss : [0.99208] Val Weighted F1 Score : [0.85597]


  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

Epoch [70], Train Loss : [0.60159] Val Loss : [1.00473] Val Weighted F1 Score : [0.85667]


In [26]:
save_path = './checkpoints/' 
torch.save(infer_model, save_path + 'densnet161_aug_sampling_smoothing_submit(70epoch)_2.pth')

## 11. Test

### 1) test img load

In [27]:
test = pd.read_csv('./data/open/test.csv')

In [28]:
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


### 2) 경로수정

In [29]:
test['img_path1'] = test.img_path.apply(lambda x : './data/open' + x[1:])

In [30]:
test

,id,img_path,img_path1
0,TEST_000,./test/000.png,./data/open/test/000.png
1,TEST_001,./test/001.png,./data/open/test/001.png
2,TEST_002,./test/002.png,./data/open/test/002.png
3,TEST_003,./test/003.png,./data/open/test/003.png
4,TEST_004,./test/004.png,./data/open/test/004.png
...,...,...,...
787,TEST_787,./test/787.png,./data/open/test/787.png
788,TEST_788,./test/788.png,./data/open/test/788.png
789,TEST_789,./test/789.png,./data/open/test/789.png
790,TEST_790,./test/790.png,./data/open/test/790.png


### 3) 인퍼런스 

In [31]:
test_dataset = CustomDataset(test['img_path1'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [32]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [33]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/50 [00:00<?, ?it/s]

### 4) csv 저장

In [34]:
submit = pd.read_csv('./data/open/sample_submission.csv')

In [35]:
submit

,id,label
0,TEST_000,훼손
1,TEST_001,훼손
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,훼손
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,훼손
790,TEST_790,훼손


In [36]:
submit['label'].value_counts()

label
훼손    792
Name: count, dtype: int64

In [37]:
submit['label'] = preds

In [38]:
submit.to_csv('./densnet161_aug_sampling_smoothing_submit(70epoch)_2.csv', index=False)

## test

In [39]:
check_submit = pd.read_csv('./densnet161_aug_sampling_smoothing_submit(70epoch)_2.csv')

In [40]:
check_submit

,id,label
0,TEST_000,오염
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,오염
789,TEST_789,오염
790,TEST_790,오염


In [41]:
check_submit['label'].value_counts()

label
훼손         331
오염         279
꼬임          30
면불량         23
터짐          20
곰팡이         17
오타공         16
몰딩수정        15
걸레받이수정      14
이음부불량        9
들뜸           7
가구수정         6
피스           6
창틀,문틀수정      6
석고수정         5
울음           4
녹오염          3
반점           1
Name: count, dtype: int64

In [42]:
#check_submit['label'] = le.fit_transform(check_submit['label'])

In [43]:
#check_submit

In [44]:
#check_submit['label'].value_counts()

In [45]:
#check_submit['label'] = le.inverse_transform(check_submit['label'])

In [46]:
#check_submit

In [47]:
#check_submit['label'].value_counts()